In [1]:
import pandas as pd
import numpy as np

In [2]:
loan_data_inputs_pd_temp=pd.read_csv('loan_data_inputs_pd_temp.csv')

In [3]:
loan_data_inputs_pd=pd.read_csv('loan_data_inputs_pd.csv')

In [4]:
loan_data_inputs_pd_temp.head()

,Unnamed: 0,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_VA,...,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
0,427211,1,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
1,206088,0,0,1,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0
2,136020,1,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
3,412305,0,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,36159,0,0,1,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0


In [5]:
from sklearn import linear_model
import scipy.stats as stat

class LogisticRegression_with_p_values:
    
    def __init__(self,*args,**kwargs):
        self.model = linear_model.LogisticRegression(*args,**kwargs)

    def fit(self,X,y):
        self.model.fit(X,y)
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X / denom).T,X)
        Cramer_Rao = np.linalg.inv(F_ij)
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.model.coef_[0] / sigma_estimates
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores]
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = p_values

In [6]:
import pickle

In [7]:
reg_pd = pickle.load(open('pd_model.sav', 'rb'))
# We import the PD model, stored in the 'pd_model.sav' file.

In [8]:
#Now we have imported all three model PD, EL and LGD.

In [9]:
loan_data_inputs_pd_temp.set_index('Unnamed: 0', inplace=True)

In [10]:
#We  have calculated loan_data_inputs_pd_temp directly so they are arranged.
#So we can just calculate PD as new column in the dataframe.

In [11]:
reg_pd.model.predict_proba(loan_data_inputs_pd_temp)[: ][: , 0]
# We apply the PD model to caclulate estimated default probabilities.

array([0.02958458, 0.09214655, 0.03735817, ..., 0.02678623, 0.04020793,
       0.04763329])

In [12]:
#predict_proba returns arrays containing many arrays.
#Each lower level or second level array conatins 2 class, first is probability of default (0) and second is probability of non-
#default (1)
#Since we want to calculate the probability of default so we need the first number of each array. So, column with index 0

In [13]:
#LGD and EAD are in the loan_data_preprocessed
#PD is in loan_data_inputs_pd
#Both this dataframe have similar index so when we concate them then they  will be matched properly.

In [14]:
# Import data.
loan_data_preprocessed = pd.read_csv("loan_data_preprocessed1.csv")

C:\Users\Sonit Prasad\AppData\Local\Temp\ipykernel_8440\4057527393.py:2: DtypeWarning: Columns (22,50) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data_preprocessed = pd.read_csv("loan_data_preprocessed1.csv")


In [ ]:
# Import data.
#loan_data_preprocessed = pd.read_csv("D:\Projects\loan_data_2007_2014_preprocessed.csv")

In [15]:
loan_data_inputs_pd['PD'] = reg_pd.model.predict_proba(loan_data_inputs_pd_temp)[: ][: , 0]
# We apply the PD model to caclulate estimated default probabilities.

In [16]:
loan_data_inputs_pd['PD'].head()

0    0.029585
1    0.092147
2    0.037358
3    0.204335
4    0.200837
Name: PD, dtype: float64

In [17]:
loan_data_inputs_pd['PD'].describe()
# Shows some descriptive statisics for the values of a column.

count    466285.000000
mean          0.109306
std           0.070917
min           0.007313
25%           0.056063
50%           0.093492
75%           0.146558
max           0.635838
Name: PD, dtype: float64

In [18]:
loan_data_preprocessed_new = pd.concat([loan_data_preprocessed, loan_data_inputs_pd], axis = 1)
# We concatenate the dataframes where we calculated LGD and EAD and the dataframe where we calculated PD along the columns.

In [19]:
loan_data_preprocessed_new.shape

(466285, 542)

In [20]:
loan_data_preprocessed_new.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,...,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86,PD
0,0,0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,...,0,0,1,0,0,0,0,0,0,0.029585
1,1,1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,...,0,0,1,0,0,0,0,0,0,0.092147
2,2,2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,...,0,0,1,0,0,0,0,0,0,0.037358
3,3,3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,...,0,0,1,0,0,0,0,0,0,0.204335
4,4,4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,...,1,0,1,0,0,0,0,0,0,0.200837


In [21]:
loan_data_preprocessed_new.to_csv('loan_data_preprocessed_new.csv')